<a href="https://colab.research.google.com/github/joerowelll/COMP0132/blob/main/structureFromMotion/colmap_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COLMAP
This notebook is a command line interface (CLI) implementation of COLMAP Structure from Motion software; to output both sparse and dense reconstructions.

If the execution result of running the code cell below is 'Not connected to a GPU', you can change the runtime by going to Runtime > Change runtime type in the menu to enable a GPU accelerator, and then re-execute the code cell.



In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# Installation

In [ ]:
!sudo apt-get install \
    git \
    cmake \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-regex-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libfreeimage-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libcgal-qt5-dev \
    libmetis-dev
!source ~/.bashrc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libboost-filesystem-dev is already the newest version (1.65.1.0ubuntu1).
libboost-filesystem-dev set to manually installed.
libboost-program-options-dev is already the newest version (1.65.1.0ubuntu1).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.65.1.0ubuntu1).
libboost-system-dev set to manually installed.
libboost-graph-dev is already the newest version (1.65.1.0ubuntu1).
libboost-graph-dev set to manually installed.
libboost-regex-dev is already the newest version (1.65.1.0ubuntu1).
libboost-regex-dev set to manually installed.
libboost-test-dev is already the newest version (1.65.1.0ubuntu1).
libboost-test-dev set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
git is already the newest version (1:2.17.1-1ubuntu0.11).
qtbase5-

## Install Ceres-solver (takes 10~20 minutes...)

In [ ]:
!sudo apt-get install libatlas-base-dev libsuitesparse-dev
!git clone https://ceres-solver.googlesource.com/ceres-solver
%cd ceres-solver
!git checkout 2.0.0
# Checkout the latest release
%mkdir build
%cd build
!cmake .. -DBUILD_TESTING=OFF -DBUILD_EXAMPLES=OFF
!make
!sudo make install

## Install colmap (takes another 10~20 minutes...)

In [ ]:
!git clone https://github.com/colmap/colmap
%cd colmap
!git checkout dev
%mkdir build
%cd build
!cmake ..
!make
!sudo make install
!CC=/usr/bin/gcc-6 CXX=/usr/bin/g++-6 cmake ..

fatal: destination path 'colmap' already exists and is not an empty directory.
/content/ceres-solver/build/colmap/build/colmap
Already on 'dev'
Your branch is up to date with 'origin/dev'.
mkdir: cannot create directory ‘build’: File exists
/content/ceres-solver/build/colmap/build/colmap/build
-- Found AMD headers in: /usr/include/suitesparse
-- Found AMD library: /usr/lib/x86_64-linux-gnu/libamd.so
-- Found CAMD headers in: /usr/include/suitesparse
-- Found CAMD library: /usr/lib/x86_64-linux-gnu/libcamd.so
-- Found CCOLAMD headers in: /usr/include/suitesparse
-- Found CCOLAMD library: /usr/lib/x86_64-linux-gnu/libccolamd.so
-- Found CHOLMOD headers in: /usr/include/suitesparse
-- Found CHOLMOD library: /usr/lib/x86_64-linux-gnu/libcholmod.so
-- Found COLAMD headers in: /usr/include/suitesparse
-- Found COLAMD library: /usr/lib/x86_64-linux-gnu/libcolamd.so
-- Found SPQR headers in: /usr/include/suitesparse
-- Found SPQR library: /usr/lib/x86_64-linux-gnu/libspqr.so
-- Found Config he

Next, we need to prepare the images to run colmap.
First, create a folder in your google drive and a subfolder named `images`, and put your images inside.

## Mount your drive (to access data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


## Clone LLFF util

In [ ]:
%cd /content
!git clone https://github.com/Fyusion/LLFF

/content
Cloning into 'LLFF'...
remote: Enumerating objects: 774, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 774 (delta 7), reused 20 (delta 5), pack-reused 748
Receiving objects: 100% (774/774), 31.94 MiB | 26.17 MiB/s, done.
Resolving deltas: 100% (409/409), done.


# Run COLMAP! (depending on number of images, this takes 10~20 minutes)

In [ ]:
dataset='brighton' #@param ['brighton','louvre']
# %cd /content/LLFF
# # change the path below to your data folder (the folder containing the `images` folder)
# !python imgs2poses.py "/content/drive/My Drive/COMP0132/{dataset}/"

# The project folder must contain a folder "images" with all the images.

!DATASET_PATH=/content/drive/MyDrive/COMP0132/{dataset}

!colmap feature_extractor \
 --database_path $DATASET_PATH/database.db \
  --image_path $DATASET_PATH/images

!colmap exhaustive_matcher \
  --database_path $DATASET_PATH/database.db

!mkdir $DATASET_PATH/sparse

!colmap mapper \
   --database_path $DATASET_PATH/database.db \
   --image_path $DATASET_PATH/images \
   --output_path $DATASET_PATH/sparse

!mkdir $DATASET_PATH/dense

!colmap image_undistorter \
   --image_path $DATASET_PATH/images \
   --input_path $DATASET_PATH/sparse/0 \
   --output_path $DATASET_PATH/dense \
   --output_type COLMAP \
   --max_image_size 2000

!colmap patch_match_stereo \
   --workspace_path $DATASET_PATH/dense \
   --workspace_format COLMAP \
   --PatchMatchStereo.geom_consistency true

!colmap stereo_fusion \
   --workspace_path $DATASET_PATH/dense \
   --workspace_format COLMAP \
   --input_type geometric \
   --output_path $DATASET_PATH/dense/fused.ply

!colmap poisson_mesher \
   --input_path $DATASET_PATH/dense/fused.ply \
   --output_path $DATASET_PATH/dense/meshed-poisson.ply

!colmap delaunay_mesher \
   --input_path $DATASET_PATH/dense \
   --output_path $DATASET_PATH/dense/meshed-delaunay.ply


/bin/bash: colmap: command not found
/bin/bash: colmap: command not found
mkdir: cannot create directory ‘/sparse’: File exists
/bin/bash: colmap: command not found
mkdir: cannot create directory ‘/dense’: File exists
/bin/bash: colmap: command not found
/bin/bash: colmap: command not found
/bin/bash: colmap: command not found
/bin/bash: colmap: command not found
/bin/bash: colmap: command not found


After running colmap, you will get a `poses_bounds.npy` file under your data folder, once you got that, you're ready to train!

In [ ]:
# Display mesh .ply 
! pip install open3d-python
import numpy as np
from open3d import *
cloud_poisson = read_point_cloud("meshed-poisson.ply") # Read the point cloud
cloud_delaunay = read_point_cloud("meshed-delaunay.ply") # Read the point cloud
cloud_fused = read_point_cloud("fused.ply")

draw_geometries([cloud_poisson]) # Visualize the poisson point cloud   


